In [115]:
!pip install untokenize --user

  Running setup.py bdist_wheel for untokenize ... done
  Stored in directory: /home/u25073/.cache/pip/wheels/f4/a0/63/012f35213063bcd1d92a1bb5f4a570fec73e2a0a335a9a7295
Successfully built untokenize
You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import warnings
import numpy as np
import string
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import keras
import os
import re
from collections import defaultdict

from textblob import TextBlob, Word
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from untokenize import untokenize

from sklearn import random_projection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection, naive_bayes, svm

from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer, word_tokenize

from mosestokenizer import MosesTokenizer, MosesDetokenizer
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, CuDNNGRU, CuDNNLSTM, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten, Reshape
from keras.layers import Conv1D, Conv2D, Convolution2D, MaxPool2D, MaxPooling2D, Flatten, Reshape, BatchNormalization, Concatenate
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model

from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam
from keras.regularizers import l1, l2

# from tensorflow.contrib.cudnn_rnn.python.ops import cudnn_rnn_ops

from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling1D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Model
from keras import metrics
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

stop_words = set(stopwords.words('english') + list(string.punctuation))

warnings.filterwarnings('ignore')

Using TensorFlow backend.


## Data Reading

In [20]:
Train = pd.read_csv("./data/train.csv")
Test = pd.read_csv("./data/test.csv")
Valid = pd.read_csv("./data/valid.csv")

#################################################################
# Train = Train.iloc[0:10000,:]
#################################################################

In [21]:
Train.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,dsvFUqrhytVmKXCW7bKNhA,0,2013-03-10 01:24:07,0,58nqw-MdO6EDACPaKDM59Q,5.0,"As a student, by back and neck are under const...",0,1NImHsg1kc76n_cQyTm0bg
1,6YrO-hJNof4wsx4f0YQ8yg,0,2014-12-22 17:06:23,0,eVeQtMGaB5tdCH0hKJwxKw,3.0,Stayed here for a football game at University ...,0,Oi0G3jFm2jtG2W02dZTdEQ
2,LtyoPfxpvcF_9e9wMoUi0w,0,2016-04-25 09:55:57,0,AHMHUbq0eAUOuOPFoeO5iw,4.0,"Very good salads, generous portions. I either ...",1,4jbz7cOVuV_Q7v2b3pNrLw
3,QzvLnOqwH6BIY_jCOvzuQQ,0,2017-04-06 04:18:48,0,8RDvwneSMsbpmi5UgGq60A,1.0,The experience I had with this company growing...,0,gJgPs0QXE587T9SIR6NCdQ
4,tPaweigPsXacvQT8daYT4g,0,2017-03-22 15:38:59,0,IboCoxoL0IFtJnsGPHCugg,5.0,"Easiest furniture purchase, with a great price...",0,zp8XLBJmgw55ZTlp5raK7Q


In [24]:
train = Train[['text','stars']]
test = Valid[['text','stars']]
df = pd.concat([train, test])
df = df.reset_index(drop=True)

In [25]:
print("train_data ", train['text'].shape)
print("train_labels ", train['stars'].shape)
print("_"*100)
print("test_data ", test['text'].shape)
print("test_labels ", test['stars'].shape)
print("_"*100)


train_data  (100000,)
train_labels  (100000,)
____________________________________________________________________________________________________
test_data  (10000,)
test_labels  (10000,)
____________________________________________________________________________________________________


In [26]:
df.head()

,text,stars
0,"As a student, by back and neck are under const...",5.0
1,Stayed here for a football game at University ...,3.0
2,"Very good salads, generous portions. I either ...",4.0
3,The experience I had with this company growing...,1.0
4,"Easiest furniture purchase, with a great price...",5.0


## Basic Features

In [27]:
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

def features(df):
    ## Number of Words
    df['word_count'] = df['text'].apply(lambda x: len(str(x).split(" "))) 
    ## Number of characters
    df['char_count'] = df['text'].str.len() 
    ## Average Word Length
    df['avg_word'] = df['text'].apply(lambda x: avg_word(x))
    ## Number of stopwords
    df['stopwords'] = df['text'].apply(lambda x: len([x for x in x.split() if x in stop_words]))
    ## Number of special characters
    df['hastags'] = df['text'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
    ##Number of numerics
    df['numerics'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
    ## Number of Uppercase words
    df['upper'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

In [28]:
features(df)

In [29]:
df.head()

,text,stars,word_count,char_count,avg_word,stopwords,hastags,numerics,upper
0,"As a student, by back and neck are under const...",5.0,48,295,5.166667,14,0,0,2
1,Stayed here for a football game at University ...,3.0,37,218,4.918919,13,0,0,0
2,"Very good salads, generous portions. I either ...",4.0,19,106,4.631579,4,0,0,2
3,The experience I had with this company growing...,1.0,134,716,4.343284,55,0,2,7
4,"Easiest furniture purchase, with a great price...",5.0,19,122,5.473684,6,0,0,0


## Pre-processing

In [30]:
def pre_processing(df):
    
    ## Lower case
    df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    ## Removing Punctuation
    df['text'] = df['text'].str.replace('[^\w\s]','')
    ## Removing Stopwords
    df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))
    ## Spelling correction (time-required)
#     df['text'] = df['text'].apply(lambda x: str(TextBlob(x).correct()))
    

In [31]:
pre_processing(df)

In [32]:
df['text'].head()

0    student back neck constant strain dr serrick m...
1    stayed football game university phoenix stadiu...
2    good salads generous portions either get mexic...
3    experience company growing awesome least memor...
4    easiest furniture purchase great price deliver...
Name: text, dtype: object

### Rare words removal

In [33]:
freq = pd.Series(' '.join(df['text']).split()).value_counts()[-10:]
freq

syed               1
entropy            1
safetydrwellish    1
waterhole          1
terres             1
looot              1
troublemade        1
onback             1
chipsguac          1
swizzling          1
dtype: int64

In [34]:
freq = list(freq.index)
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

tokenizer1 = Tokenizer(num_words=1000)
tokenizer1.fit_on_texts(df['text'])
vocab = tokenizer1.word_index

x_train_word_ids = tokenizer1.texts_to_sequences(train['text'])
x_test_word_ids = tokenizer1.texts_to_sequences(test['text'])

x_train_padded_seqs = pad_sequences(x_train_word_ids, maxlen=100)
x_test_padded_seqs = pad_sequences(x_test_word_ids, maxlen=100)

vectorizer = CountVectorizer(max_features=100, lowercase=True, analyzer='word',
                             stop_words='english', ngram_range=(1, 2))

X_train = vectorizer.fit_transform(train['text']).toarray()
y = Train_y.values

## Stemming and Lemmatization

### Tokenization

In [38]:
df['text'] = [word_tokenize(entry) for entry in df['text']]

### POS tagging

In [128]:
tag_map = defaultdict(lambda : wordnet.NOUN)
tag_map['J'] = wordnet.ADJ
tag_map['V'] = wordnet.VERB
tag_map['R'] = wordnet.ADV


In [126]:
def lemmatization(df):
    for index,entry in enumerate(df['text']):
        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
        # The final processed set of words for each iteration will be stored in 'text_final'
        df.loc[index,'text_final'] = str(Final_words)

In [81]:
lemmatization(df)

In [82]:
df.to_csv("pre_processed_df.csv", index=False)

## Train_test_split

In [2]:
df = pd.read_csv("./pre_processed_df.csv")

In [3]:
df

,text,stars,word_count,char_count,avg_word,stopwords,hastags,numerics,upper,text_final
0,"['student', 'back', 'neck', 'constant', 'strai...",5.0,48,295,5.166667,14,0,0,2,"['student', 'back', 'neck', 'constant', 'strai..."
1,"['stayed', 'football', 'game', 'university', '...",3.0,37,218,4.918919,13,0,0,0,"['stay', 'football', 'game', 'university', 'ph..."
2,"['good', 'salads', 'generous', 'portions', 'ei...",4.0,19,106,4.631579,4,0,0,2,"['good', 'salad', 'generous', 'portion', 'eith..."
3,"['experience', 'company', 'growing', 'awesome'...",1.0,134,716,4.343284,55,0,2,7,"['experience', 'company', 'grow', 'awesome', '..."
4,"['easiest', 'furniture', 'purchase', 'great', ...",5.0,19,122,5.473684,6,0,0,0,"['easy', 'furniture', 'purchase', 'great', 'pr..."
5,"['great', 'hotel', 'clean', 'dont', 'like', 'p...",1.0,54,293,4.444444,18,0,0,0,"['great', 'hotel', 'clean', 'dont', 'like', 'p..."
6,"['café', 'coffee', 'actually', 'important', 'w...",1.0,16,85,4.375000,6,0,0,0,"['café', 'coffee', 'actually', 'important', 'b..."
7,"['impressed', 'ever', 'wanting', 'try', 'place...",2.0,137,683,3.992701,60,0,0,6,"['impressed', 'ever', 'want', 'try', 'place', ..."
8,"['first', 'night', 'vegas', 'wanted', 'somethi...",5.0,73,382,4.246575,31,0,0,0,"['first', 'night', 'vega', 'want', 'something'..."
9,"['21', 'reviews', '12', '1star', 'variety', 'c...",1.0,405,2318,4.666667,160,0,6,10,"['review', 'variety', 'coincidence', 'shock', ..."


In [4]:
Train_x, Test_x, Train_y, Test_y = model_selection.train_test_split(
    df['text_final'], df['stars'], test_size=(1/11))

In [5]:
# Label
ohe = OneHotEncoder(sparse=False)
y_ohe_train = ohe.fit_transform(Train_y.values.reshape(-1, 1))
y_ohe_test = ohe.fit_transform(Test_y.values.reshape(-1, 1))

In [6]:
print("train_data ", Train_x.shape)
print("train_labels ", Train_y.shape)
print("_"*100)
print("test_data ", Test_x.shape)
print("test_labels ", Test_y.shape)
print("_"*100)

train_data  (100000,)
train_labels  (100000,)
____________________________________________________________________________________________________
test_data  (10000,)
test_labels  (10000,)
____________________________________________________________________________________________________


## TFIDF

In [138]:
tfidf = TfidfVectorizer(max_features=100, lowercase=True, analyzer='word',
                        stop_words='english', ngram_range=(1, 2))

train_vect = tfidf.fit_transform(Train_x)
test_vect = tfidf.fit_transform(Test_x)


In [ ]:
vocab = tfidf.vocabulary_

In [142]:
# Show the Model as a pandas DataFrame
feature_names = tfidf.get_feature_names()
train_vectdf = pd.DataFrame(train_vect, columns = feature_names)
train_vectdf.head()

,amaze,area,ask,awesome,bad,bar,best,big,bit,burger,...,use,visit,wait,walk,want,wasnt,way,work,worth,year
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.263863,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.491847,0.0,0.0,0.0,0.0,0.000000,0.000000
4,0.0,0.0,0.115995,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.114035,0.126155


## PCA

In [ ]:
from sklearn.decomposition import PCA

n_components = 30
pca = PCA(n_components)
colname = []
for i in range(0,n_components):
    colname.append('PCA'+str(i+1))
    
principalComponents = pca.fit_transform(train_vect)
principalDf = pd.DataFrame(data = principalComponents
             , columns = [colname])

## Word Index

In [91]:
tokenizer = Tokenizer(100)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 124556 unique tokens.


## Glove

In [18]:
from gensim.scripts.glove2word2vec import glove2word2vec

# glove_input_file = './glove.6B.100d.txt'
# word2vec_output_file = './glove.6B.100d.txt.word2vec'
# glove2word2vec(glove_input_file, word2vec_output_file)

from gensim.models import KeyedVectors 

filename = './glove.6B.100d.txt.word2vec'
embeddings_index = KeyedVectors.load_word2vec_format(filename, binary=False)

In [45]:
MAX_NUM_WORDS = 100000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 100

In [67]:
embedding_matrix = np.random.random((MAX_NUM_WORDS + 1, EMBEDDING_DIM))
for word, i in word_index:
    if i < MAX_NUM_WORDS:
        embedding_vector = embeddings_index[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

ValueError: too many values to unpack (expected 2)

In [ ]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

## Models

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=5, gamma='auto')
SVM.fit(train_vect,Train_y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(test_vect)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_y)*100)


In [ ]:
predictions_SVM

In [ ]:
classifierGNB = GaussianNB()
classifierGNB.fit(train_vect, Train_y)
GNB_y_train_pred = classifierGNB.predict(train_vect)
GNB_y_test_pred = classifierGNB.predict(test_vect)

print("GaussianNB Accuracy (Train):",metrics.accuracy_score(Train_y, GNB_y_train_pred))
print("GaussianNB Accuracy (Test):",metrics.accuracy_score(Test_y, GNB_y_test_pred))

# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(train_vect, Train_y)
MGNB_y_train_pred = classifierGNB.predict(train_vect)
MGNB_y_test_pred = clf.predict(test_vect)

print("MultinomialNB Accuracy (Train):",metrics.accuracy_score(Train_y, MGNB_y_train_pred))
print("MultinomialNB Accuracy (Test):",metrics.accuracy_score(Test_y, MGNB_y_test_pred))



In [ ]:
MGNB_y_test_pred

In [ ]:
# Hyperparameters
input_length = 100
embedding_size = 100
MAX_SEQUENCE_LENGTH = 30
hidden_size = 100
batch_size = 100
dropout_rate = 0.5
filters = 100
kernel_sizes = [3, 6, 9]
padding = 'valid'
activation = 'relu'
strides = 1
pool_size = 2
learning_rate = 0.1
total_epoch = 10
output_size = 5

In [ ]:
# Creating Callbacks
# ModelCheckpoints is used to save the model after every epoch
# EarlyStopping is used to stop training when the validation loss has not improved after 2 epochs
# Tensorboard is used tovisualize dynamic graphs of the training and test metrics
cbks = [callbacks.ModelCheckpoint(filepath='./checkpoint_model.h5', monitor='val_loss', save_best_only=True),
            callbacks.EarlyStopping(monitor='val_loss', patience=2),callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)]

In [93]:
# Final Model Architecture

model = Sequential()
model.add(Embedding(input_dim=100000, output_dim=embedding_size,
                    input_length=input_length, dropout=0.2))
model.add(Conv1D(filters=filters, kernel_size=3, padding='same',
                 activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Conv1D(filters=filters, kernel_size=6, padding='same',
                 activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Conv1D(filters=filters, kernel_size=9, padding='same',
                 activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(MaxPooling1D(pool_size=((input_length-kernel_sizes[2])//strides+1)))
# 1 layer of 100 units in the hidden layers of the LSTM cells
model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(output_size, activation='softmax',
                kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 100)          10000000  
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 100, 100)          30100     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 100, 100)          60100     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 100, 100)          90100     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 1, 100)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 505       
Total para

In [205]:
# Final model2 Architecture

model2 = Sequential()
model2.add(Embedding(input_dim=100000, output_dim=embedding_size,
                     input_length=input_length, dropout=0.2))
model2.add(Conv1D(filters=filters, kernel_size=3, padding='same',
                  activation='relu'))
model2.add(Conv1D(filters=filters, kernel_size=6, padding='same',
                  activation='relu'))
model2.add(Conv1D(filters=filters, kernel_size=9, padding='same',
                  activation='relu'))
model2.add(MaxPooling1D(pool_size=((input_length-kernel_sizes[2])//strides+1)))
# 1 layer of 100 units in the hidden layers of the LSTM cells
model2.add(LSTM(100))
model2.add(Dense(output_size, activation='softmax'))

optimizer = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

model2.compile(loss='categorical_crossentropy',
               optimizer=optimizer, metrics=['accuracy'])
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 100, 100)          10000000  
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 100, 100)          30100     
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 100, 100)          60100     
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 100, 100)          90100     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 1, 100)            0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 505       
Total para

In [111]:
# New model
# YOUR CODE HERE
x = Input(shape=(100, ))

# embedding layer and dropout
# YOUR CODE HERE
e = Embedding(input_dim=100, output_dim=embedding_size, input_length=input_length)(x)
e_d = Dropout(dropout_rate)(e)

# construct the sequence tensor for CNN
# YOUR CODE HERE
e_d = Reshape((input_length, embedding_size, 1))(e_d)

# CNN layers
conv_blocks = []
for kernel_size in kernel_sizes:
    # YOUR CODE HERE
    conv = Conv2D(filters=filters, kernel_size=(kernel_size, embedding_size), padding=padding, activation=activation, strides=(strides, strides), activity_regularizer=l1(0.001))(e_d)
    maxpooling = MaxPool2D(pool_size=((input_length-kernel_size)//strides+1, 1))(conv)
    faltten = Flatten()(maxpooling)
    conv_blocks.append(faltten)

# concatenate CNN results
# YOUR CODE HERE
c = Concatenate()(conv_blocks) if len(kernel_sizes) > 1 else conv_blocks[0]
c_d = Dropout(dropout_rate)(c)

# dense layer
# YOUR CODE HERE
L = LSTM(100)
d = Dense(hidden_size, activation=activation, activity_regularizer=l1(0.001))(c_d)


# output layer
# YOUR CODE HERE
y = Dense(output_size, activation='softmax', activity_regularizer=l1(0.001))(d)

# build your own model
# YOUR CODE HERE
model_CNN = Model(x, y)

# SGD optimizer with momentum
optimizer = SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=True)

# compile model
model_CNN.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model_CNN.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 100, 100)     10000       input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 100, 100)     0           embedding_10[0][0]               
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 100, 100, 1)  0           dropout_1[0][0]             

In [112]:
model_CNN.fit(train_vect, y_ohe_train, epochs=1,
          verbose=1, batch_size=32, callbacks=cbks)

train_score = model_CNN.evaluate(
    train_vect, y_ohe_train, batch_size=batch_size)
test_score = model_CNN.evaluate(
    test_vect, y_ohe_test, batch_size=batch_size)

print('Training Loss: {}\n Training Accuracy: {}\n'
      'Testng Loss: {}\n Testing accuracy: {}'.format(
          train_score[0], train_score[1],
          test_score[0], test_score[1]))

Epoch 1/1
10000/10000 [==============================] - 12s 1ms/step
Training Loss: 1.531957049012184
 Training Accuracy: 0.43957999765872957
Testng Loss: 1.5277322494983674
 Testing accuracy: 0.4405999979376793


In [207]:
model2.fit(train_vect, y_ohe_train, epochs=1,
          verbose=1, batch_size=32, callbacks=cbks)

train_score = model2.evaluate(
    train_vect, y_ohe_train, batch_size=batch_size)
test_score = model2.evaluate(
    test_vect, y_ohe_test, batch_size=batch_size)

print('Training Loss: {}\n Training Accuracy: {}\n'
      'Testng Loss: {}\n Testing accuracy: {}'.format(
          train_score[0], train_score[1],
          test_score[0], test_score[1]))

Epoch 1/1
10000/10000 [==============================] - 7s 698us/step
Training Loss: 1.4426082400083542
 Training Accuracy: 0.43957999765872957
Testng Loss: 1.438023966550827
 Testing accuracy: 0.4405999979376793


## Visualization of loss and accuracy

In [101]:
## Visualizing Loss

plt.switch_backend('agg')
%matplotlib inline

fig1 = plt.figure()
plt.plot(model.model['loss'],'r',linewidth=3.0)
plt.legend(['Training Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('CNN Loss',fontsize=16)
fig1.savefig('loss_cnn.png')
plt.show()

TypeError: 'Sequential' object is not subscriptable

<Figure size 432x288 with 0 Axes>

In [102]:
## Visualizing accuracy

fig2 = plt.figure()
plt.plot(model.model['acc'], 'r', linewidth=3.0)
plt.legend(['Training Accuracy'], fontsize=18)
plt.xlabel('Epochs ', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.title('CNN Accuracy', fontsize=16)
fig2.savefig('accuracy_cnn.png')
plt.show()

TypeError: 'Sequential' object is not subscriptable

<Figure size 432x288 with 0 Axes>

## Prediction Output

In [114]:
# predicting
test_pre = model_CNN.predict(test_vect, batch_size=batch_size)
# sub_df = pd.DataFrame()
# sub_df["review_id"] = test_id_list
# sub_df["pre"] = test_pre
# sub_df.to_csv("pre.csv", index=False)

In [115]:
test_pre

array([[0.11774056, 0.08284722, 0.13442786, 0.22003506, 0.4449493 ],
       [0.11774056, 0.08284722, 0.13442786, 0.22003506, 0.4449493 ],
       [0.11774056, 0.08284722, 0.13442786, 0.22003506, 0.4449493 ],
       ...,
       [0.11774056, 0.08284722, 0.13442786, 0.22003506, 0.4449493 ],
       [0.11774056, 0.08284722, 0.13442786, 0.22003506, 0.4449493 ],
       [0.11774056, 0.08284722, 0.13442786, 0.22003506, 0.4449493 ]],
      dtype=float32)